# nn.Conv1d
核心概念：与 Conv2D 的对比
`nn.Conv1d` 的核心思想是在一维序列上滑动一个滤波器（卷积核）。
| 特性     | nn.Conv2d (二维卷积)            | nn.Conv1d (一维卷积)                     |
|----------|----------------------------------|------------------------------------------|
| 数据类型   | 2D数据，如图像                   | 1D数据，如时间序列、文本、音频            |
| 滤波器形状 | 2D，如 (3, 3)                   | 1D，如 (3,)                             |
| 滑动方式   | 在图像的高度和宽度上滑动          | 仅在序列的长度方向上滑动                 |
| 检测目标   | 空间模式，如边缘、纹理、形状       | 序列模式 (Motifs)，如特定n-gram、波形成分 |

## 核心功能与应用场景
`nn.Conv1d` 主要用于从序列数据中提取局部模式。它的滤波器会在序列上滑动，每次只关注一小段连续的片段（由 `kernel_size` 决定），从而识别出有意义的局部特征。

主要应用场景：

- 自然语言处理 (NLP)：

    - 数据：一个句子可以表示为词向量序列。输入形状为 (`批次大小, 词向量维度, 句子长度`)。
    
    - 作用：`nn.Conv1d` 可以识别 `N-grams`（连续的N个词）。例如，一个大小为`3`的`卷积核`可以学习识别像 "I love you" 或 "New York City" 这样的短语模式。

- 时间序列分析：

    - 数据：传感器读数、股票价格、心电图 (ECG) 信号等。输入形状为 (`批次大小, 特征数, 时间步数`)。
    
    - 作用：可以识别时间序列中的特定形状或模式，如某个特定周期的价格波动、心电图中的异常波形等。

- 音频处理：

    - 数据：原始音频波形。
    
    - 作用：可以作为滤波器，提取音频信号中的特定频率成分。

- 基因组学：

    - 数据：DNA或蛋白质序列。
    
    - 作用：识别基因序列中的特定模式（Motifs）。

---
### 关键参数详解
`nn.Conv1d` 的参数与 `nn.Conv2d `非常相似，只是没有了“高度”这个维度。

In [ ]:
torch.nn.Conv1d(
    in_channels,
    out_channels,
    kernel_size,
    stride=1,
    padding=0,
    dilation=1,
    groups=1,
    bias=True,
    padding_mode='zeros'
)

- `in_channels` (整数): 输入序列中每个时间步的特征数。

    - 在NLP中，这通常是词嵌入的维度 (embedding dimension)。
    
    - 在时间序列中，这可能是多个传感器在同一时刻的读数。

- `out_channels` (整数): `滤波器`（卷积核）的`数量`，决定了输出序列的通道数。代表`模型想要学习的模式种类数量`。

- `kernel_size` (整数): 卷积核的长度。`kernel_size=3` 意味着滤波器会一次性查看序列中3个连续的时间步。

- `stride` (整数): 滤波器滑动的步长。

- `padding` (整数): 在序列的两端进行填充。`padding='same'` (需手动计算) 可以让输出序列和输入序列等长。

---
### 输入与输出的形状 (Shape)
这是理解 `nn.Conv1d` 的关键。

- 输入形状: (N, C_in, L_in)

    - N: 批次大小 (Batch Size)

    - C_in: 输入通道数 (`in_channels`)
    
    - L_in: 序列长度 (`Sequence Length`)

- 输出形状: (N, C_out, L_out)

    - N: 批次大小 (不变)

    - C_out: 输出通道数 (out_channels)
    
    - L_out: 输出序列长度 (需要计算)

# 输出序列长度计算公式

对于序列模型，输出序列长度 $L_{out}$ 的计算公式通常为：

$$
L_{out} = \left\lfloor \frac{L_{in} + 2p - k}{s} \right\rfloor + 1
$$

其中：
- $L_{in}$：输入序列长度
- $p$：填充数（padding）
- $k$：卷积核大小/窗口大小（kernel size）
- $s$：步长（stride）
- $\left\lfloor \cdot \right\rfloor$：向下取整函数

## 示例计算

假设：
- $L_{in} = 10$
- $p = 1$
- $k = 3$
- $s = 2$

则：
$$
L_{out} = \left\lfloor \frac{10 + 2 \times 1 - 3}{2} \right\rfloor + 1 = \left\lfloor \frac{9}{2} \right\rfloor + 1 = 4 + 1 = 5
$$

---
## 示例计算 (NLP场景):
假设我们有一个批次的数据，包含32个句子，每个句子最多50个词，每个词用100维的向量表示。

- 输入张量形状: (32, 100, 50)  (N=32, C_in=100, L_in=50)

- 卷积层定义: `nn.Conv1d`(`in_channels`=100, `out_channels`=256, `kernel_size`=3, `padding`=1)

我们想要学习256种不同的 `"3-gram"`模式。

- `padding`=1 是为了保持序列长度不变。

计算输出长度 `L_out`: 50
- 最终输出张量形状: (32, 256, 50)

---
### 代码示例 (文本分类)
这个例子展示了如何使用 `nn.Conv1d `对文本进行分类，这是一个非常经典的应用（`TextCNN`）。

In [ ]:
import torch
import torch.nn as nn

class TextClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, num_classes):
        super(TextClassifier, self).__init__()
        
        # 1. 嵌入层：将词索引转换为词向量
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        # 2. 卷积层：输入通道数=词向量维度，输出通道数=滤波器数量
        self.conv1d = nn.Conv1d(
            in_channels=embedding_dim, 
            out_channels=128, # 学习128种模式
            kernel_size=3, # 类似 3-grams
            padding=1
        )
        self.relu = nn.ReLU()
        
        # 3. 池化层：在时间步维度上取最大值，捕获最重要的信号
        # Global Max Pooling
        
        # 4. 全连接层：进行分类
        self.fc = nn.Linear(128, num_classes)

    def forward(self, x):
        # x 初始形状: (N, L_in) -> (批次大小, 句子长度)
        
        # 1. 经过嵌入层
        x = self.embedding(x) # -> (N, L_in, embedding_dim)
        
        # 2. 调整维度以匹配Conv1d的输入 (N, C_in, L_in)
        # PyTorch 的卷积层要求通道数在第二个维度
        x = x.permute(0, 2, 1) # -> (N, embedding_dim, L_in)
        
        # 3. 经过卷积层
        x = self.conv1d(x) # -> (N, 128, L_in)
        x = self.relu(x)
        
        # 4. 全局最大池化
        # 在序列长度维度上取最大值
        x = torch.max(x, dim=2)[0] # -> (N, 128)
        
        # 5. 经过全连接层分类
        output = self.fc(x) # -> (N, num_classes)
        return output

# --- 模型和数据设置 ---
VOCAB_SIZE = 1000  # 词汇表大小
EMBEDDING_DIM = 100 # 词向量维度
SEQ_LENGTH = 50    # 句子长度
NUM_CLASSES = 10   # 类别数
BATCH_SIZE = 32

# 创建模型实例
model = TextClassifier(VOCAB_SIZE, EMBEDDING_DIM, NUM_CLASSES)

# 创建假的输入数据 (词索引)
dummy_input = torch.randint(0, VOCAB_SIZE, (BATCH_SIZE, SEQ_LENGTH))

# 前向传播
output = model(dummy_input)

print("输入形状:", dummy_input.shape)
print("输出形状:", output.shape)

# 输出:
# 输入形状: torch.Size([32, 50])
# 输出形状: torch.Size([32, 10])